# Applied Data Science Capstone Assignment Week 3

**_Note:_** Unfortunately the table of content links won't work in Github directly. To use them please copy and paste the link below into your browser to open the notebook in **nbviewer** :


## Table of Content

1. <a href="#part1">Part 1 - Scraping and transforming table<br></a>
    1. <a href="#part1a">Import necessary libraries<br></a>
    2. <a href="#part1b">Scraping table from wikipedia and reading it into a dataframe<br></a>
    3. <a href="#part1c">Transforming table<br></a>
2. <a href="#part2">Part 2 - Adding geo-locational data<br></a>
    1. <a href="#part2a">Import necessary libraries<br></a>
    2. <a href="#part2b">Fetching geo-locational data<br></a>
    3. <a href="#part2c">Creating a new dataframe with the combined information<br></a>
3. <a href="#part3">Part 3 - Exploring and clustering neighborhoods<br></a>
    1. <a href="#part3a">Import necessary libraries<br></a>
    2. <a href="#part3b">Visualize toronto and its neighbourhoods<br></a>
    3. <a href="#part3c">Explore neighbourhoods of toronto<br></a>
    4. <a href="#part3d">Analyze the neighborhoods<br></a>
    5. <a href="#part3e">Cluster neighborhoods<br></a>

<h2 id="part1">1) Part 1 - Scraping and transforming table</h2>

<h3 id="part1a">1A) Import necessary libraries</h3>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h3 id="part1b">1B) Scraping table from wikipedia and reading it into a dataframe</h3>

0th index is taken beacuse this is the first of three tables on this page

In [2]:
df_tor = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

<h3 id="part1c">1C) Transforming table</h3>

Change the column names accordingly

In [3]:
df_tor.columns = ["PostalCode", "Borough", "Neighborhood"]

Drop rows that have no borough assigned

In [4]:
df_tor = df_tor[df_tor["Borough"] != "Not assigned"]

If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [5]:
df_tor[df_tor["Neighborhood"] == "Not assigned"]["Neighborhood"] = df_tor[df_tor["Neighborhood"] == "Not assigned"]["Borough"]

Reset index to start with 0

In [6]:
df_tor.reset_index(drop=True, inplace=True)

Neighborhoods have already been combined on the wikipedia page so there is no need to do that here

In [7]:
print("Number of unique postal codes:", df_tor["PostalCode"].nunique())
print("Number of rows:", df_tor.shape[0])

Number of unique postal codes: 103
Number of rows: 103


In [8]:
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Print the number of rows of the dataframe

In [9]:
print(f"The dataframe has {df_tor.shape[0]} rows and {df_tor.shape[1]} columns")

The dataframe has 103 rows and 3 columns


<h2 id="part2">2) Part 2 - Adding geo-locational data</h2>

<h3 id="part2a">2A) Import necessary libraries</h3>

In [10]:
#%conda install -c conda-forge geocoder

In [11]:
import geocoder  # api to get location data
import urllib.request  # library for downloading files from web

<h3 id="part2b">2B) Fetching geo-locational data</h3>

Trying to get the data from the geocoder api

In [12]:
for index in df_tor.index:
    # get postal code for that row
    postal_code = df_tor.iloc[index]["PostalCode"]
    
    print("Current index:", index, "Postal code:", postal_code)
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while lat_lng_coords is None:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    # add long- and latitude to dataframe
    df_tor.iloc[index]["Latitude"] = lat_lng_coords[0]
    df_tor.iloc[index]["Longitude"] = lat_lng_coords[1]
print("Geo data sucessfully added!")

Current index: 0 Postal code: M3A


KeyboardInterrupt: 

Since retrieving the data via the api did not work (never got past first row) the csv file will be used instead

In [13]:
url = 'http://cocl.us/Geospatial_data'
filename = 'geospatial_data.csv'
urllib.request.urlretrieve(url, filename)  # download file from url and save it locally

('geospatial_data.csv', <http.client.HTTPMessage at 0x1296c654f08>)

Read csv file into a dataframe

In [14]:
df_tor_geo = pd.read_csv(filename)
# Change column names to match other dataframe
df_tor_geo.columns = ["PostalCode", "Latitude", "Longitude"]

In [15]:
df_tor_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3 id="part2c">2C) Creating a new dataframe with the combined information</h3>

Combine the two tables in an SQL left join like manner

In [16]:
df_tor_comb = pd.merge(df_tor, df_tor_geo, how="left", on="PostalCode")

Inspect the new table

In [17]:
df_tor_comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [18]:
df_tor_comb.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h2 id="part3">3) Part 3 - Exploring and clustering neighborhoods</h2>

<h3 id="part3a">3A) Import necessary libraries</h3>

In [19]:
#%conda install -c conda-forge folium=0.5.0 --yes

In [20]:
#%conda install -c conda-forge geopy --yes

In [21]:
import numpy as np

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

<h3 id="part3b">3B) Visualize toronto and its neighbourhoods</h3>

Use geopy to get the long- and latitude of toronto

In [22]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top

In [23]:
# create map of Toronto using latitude and longitude values
lat_offset = .065 # little offset to fit it better in the window
map_toronto = folium.Map(location=[latitude+lat_offset, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor_comb['Latitude'], df_tor_comb['Longitude'], df_tor_comb['Borough'], df_tor_comb['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3 id="part3c">3C) Explore neighbourhoods of toronto</h3>

Define Foursquare credentials and version

In [24]:
CLIENT_ID = 'MK0SEV54RIMKQW3Y514Z3QLU1R24CAEYS3KUVRYX3C4QNANJ' # your Foursquare ID
CLIENT_SECRET = 'QWAKFPQFROEBEBQ1BSMJ4X0ZSAYVBDJJLCODBFLSHMXLDCPA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MK0SEV54RIMKQW3Y514Z3QLU1R24CAEYS3KUVRYX3C4QNANJ
CLIENT_SECRET:QWAKFPQFROEBEBQ1BSMJ4X0ZSAYVBDJJLCODBFLSHMXLDCPA


Define a helper function to explore all the neighborhoods

In [79]:
def getNearbyVenues(names, boroughs, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, borough, lat, lng in zip(names, boroughs, latitudes, longitudes):
        print(borough, name)
            
        # create the API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?"\
        f"&client_id={CLIENT_ID}"\
        f"&client_secret={CLIENT_SECRET}"\
        f"&v={VERSION}"\
        f"&ll={lat},{lng}"\
        f"&radius={radius}"\
        f"&limit={LIMIT}"
        
     
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            borough,
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Borough',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Store the venue information in a new dataframe

In [80]:
# define needed variables
LIMIT = 100
radius = 500

toronto_venues = getNearbyVenues(
    names=df_tor_comb["Neighborhood"],
    boroughs=df_tor_comb["Borough"],
    latitudes=df_tor_comb["Latitude"],
    longitudes=df_tor_comb["Longitude"]
)

North York Parkwoods
North York Victoria Village
Downtown Toronto Regent Park, Harbourfront
North York Lawrence Manor, Lawrence Heights
Downtown Toronto Queen's Park, Ontario Provincial Government
Etobicoke Islington Avenue, Humber Valley Village
Scarborough Malvern, Rouge
North York Don Mills
East York Parkview Hill, Woodbine Gardens
Downtown Toronto Garden District, Ryerson
North York Glencairn
Etobicoke West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Scarborough Rouge Hill, Port Union, Highland Creek
North York Don Mills
East York Woodbine Heights
Downtown Toronto St. James Town
York Humewood-Cedarvale
Etobicoke Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Scarborough Guildwood, Morningside, West Hill
East Toronto The Beaches
Downtown Toronto Berczy Park
York Caledonia-Fairbanks
Scarborough Woburn
East York Leaside
Downtown Toronto Central Bay Street
Downtown Toronto Christie
Scarborough Cedarbrae
North York Hillcrest Village
North York Bathur

Inspect the new dataframe

In [81]:
print("Shape of toronto_venus dataframe:", toronto_venues.shape)
toronto_venues.head(10)

Shape of toronto_venus dataframe: (2121, 8)


,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,North York,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
7,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
9,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Check how many venues were returned for each neighborhood

In [82]:
toronto_venues.groupby('Neighborhood').count()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Agincourt,4,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27,27
Berczy Park,58,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17,17


Get unique categories

In [83]:
print(f"There are {len(toronto_venues['Venue Category'].unique())} unique categories.")

There are 270 unique categories.


<h3 id="part3d">3D) Analyze the neighborhoods</h3>

Create one-hot tensors for each category

In [84]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot['Borough'] = toronto_venues['Borough']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print("Shape of new dataframe:", toronto_venues.shape)
toronto_onehot.head()

Shape of new dataframe: (2121, 8)


,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Group rows by neighborhood and calculate the mean of the frequency of each category

In [92]:
df_tor_comb.groupby(['Neighborhood', 'Borough']).size().reset_index().rename(columns={0:'Count'})

,Neighborhood,Borough,Count
0,Agincourt,Scarborough,1
1,"Alderwood, Long Branch",Etobicoke,1
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,1
3,Bayview Village,North York,1
4,"Bedford Park, Lawrence Manor East",North York,1
5,Berczy Park,Downtown Toronto,1
6,"Birch Cliff, Cliffside West",Scarborough,1
7,"Brockton, Parkdale Village, Exhibition Place",West Toronto,1
8,"Business reply mail Processing Centre, South C...",East Toronto,1
9,"CN Tower, King and Spadina, Railway Lands, Har...",Downtown Toronto,1


In [85]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood', 'Borough']).mean().reset_index()
print("Shape of new dataframe:", toronto_grouped.shape)
toronto_grouped

Shape of new dataframe: (94, 271)


,Neighborhood,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,Scarborough,0.000000,0.000000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.

Print the top 5 most common venues of each neighborhood

In [95]:
toronto_grouped.drop("Borough", axis=1, inplace=True)

In [96]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Pub  0.14
2             Gym  0.14
3    Skating Rink  0.14
4  Sandwich Place  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Coffee Shop  0.10
1               Bank  0.10
2  Mobile Phone Shop  0.05
3        Supermarket  0.05
4         Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.07
1       Italian Restaurant  0.07
2               Restaurant  0.07

                 venue  freq
0            Drugstore  0.33
1  Rental Car Location  0.33
2        Garden Center  0.33
3   Miscellaneous Shop  0.00
4  Moroccan Restaurant  0.00


----Northwood Park, York University----
                   venue  freq
0         Massage Studio  0.14
1                    Bar  0.14
2          Metro Station  0.14
3   Caribbean Restaurant  0.14
4  Vietnamese Restaurant  0.14


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
                 venue  freq
0       Baseball Field   1.0
1    Accessories Store   0.0
2   Miscellaneous Shop   0.0
3  Moroccan Restaurant   0.0
4  Monument / Landmark   0.0


----Parkdale, Roncesvalles----
                         venue  freq
0               Breakfast Spot  0.14
1                    Gift Shop  0.14
2                  Coffee Shop  0.07
3  Eastern European Restaurant  0.07
4                Movie Theater  0.07


----Parkview Hill, Woo

First, write a function to sort the venues in descending order then store it in a dataframe

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe to display the top 10 venues for each neighborhood

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pub,Sandwich Place,Gym,Airport Service,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Deli / Bodega,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Restaurant,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Hobby Shop,Juice Bar,Fast Food Restaurant,Butcher,Café


<h3 id="part3e">3E) Cluster neighborhoods</h3>

Cluster the neigborhoods into 5 clusters with k-means

In [98]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Merge the new dataframe with the original dataframe

In [108]:
# add clustering labels
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except ValueError:
    pass # pass if already exists

toronto_merged = df_tor_comb.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(axis=0, inplace=True) # Drop rows that have no cluster assigned
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype("int8")
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Food & Drink Shop,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Beer Store,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Concert Hall,Dim Sum Restaurant,Ethiopian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café


Visualize the clusters on a map

In [110]:
# create map
map_clusters = folium.Map(location=[latitude+lat_offset, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(f"{poi} Cluster {cluster}", parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine the clusters

Define helper function to get a better understanding of the clusters

In [178]:
def get_venue_freq(cluster_num):
    df_cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_num, toronto_merged.columns[list(range(6, toronto_merged.shape[1]))]]
    freq_counts = []
    weights = np.linspace(1, 0, df_cluster.shape[1]) # weight the each count based on frequency
    # loop through columns and store frequency count in list
    for i, w in enumerate(weights):
        val_count = df_cluster.iloc[:, i].value_counts() * w  # multiply by weight
        freq_counts.append(val_count)
    df_freq_counts = pd.DataFrame(freq_counts).fillna(0) # create dataframe and replace NaN values with 0
    df_freq_counts = df_freq_counts.agg("sum", axis=0) * 100 / df_freq_counts.agg("sum", axis=0).agg("sum", axis=0) # sum rows and convert to percentage
    return df_freq_counts.sort_values(ascending=False) # sort

#### Cluster 1  
Using the helper function we can determine what types of venues define this cluster.  
As we can see in this cluster many venues are coffe places.

In [179]:
get_venue_freq(cluster_num=0).head()

Coffee Shop       8.967320
Café              4.941176
Restaurant        3.398693
Pizza Place       2.901961
Sandwich Place    2.745098
dtype: float64

In [115]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Beer Store,Shoe Store
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Concert Hall,Dim Sum Restaurant,Ethiopian Restaurant
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café
7,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Restaurant,Beer Store,Sporting Goods Shop,Italian Restaurant,Dim Sum Restaurant,Café,Caribbean Restaurant
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Gym / Fitness Center,Intersection,Bank,Athletics & Sports,Gastropub,Café,Pharmacy,Dumpling Restaurant,Drugstore
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Theater,Fast Food Restaurant,Plaza
10,Glencairn,Bakery,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
12,"Rouge Hill, Port Union, Highland Creek",Moving Target,Bar,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dessert Shop
13,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Restaurant,Beer Store,Sporting Goods Shop,Italian Restaurant,Dim Sum Restaurant,Café,Caribbean Restaurant


#### Cluster 2  
As we can see this cluster is very small and the top 10 venues for those neigborhoods are pretty much identical

In [180]:
get_venue_freq(cluster_num=1).head()

Baseball Field    20.000000
Yoga Studio       17.777778
Donut Shop        15.555556
Diner             13.333333
Discount Store    11.111111
dtype: float64

In [167]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"Humberlea, Emery",Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Fast Food Restaurant
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Fast Food Restaurant


#### Cluster 3  
This cluster seems the be heavily defined by parks but also diners and smaller stores

In [181]:
get_venue_freq(cluster_num=2).head()

Park                 19.444444
Diner                 9.722222
Discount Store        7.500000
Convenience Store     6.666667
Drugstore             5.555556
dtype: float64

In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
21,Caledonia-Fairbanks,Park,Women's Store,Pool,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
35,"East Toronto, Broadview North (Old East York)",Park,Convenience Store,Coffee Shop,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
61,Lawrence Park,Park,Bus Line,Swim School,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Falafel Restaurant
64,Weston,Convenience Store,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
66,York Mills West,Park,Bar,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
85,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Playground,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
91,Rosedale,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


#### Cluster  4  
There is only a single neighborhood in this cluster

In [183]:
get_venue_freq(cluster_num=3).head()

Restaurant          20.000000
Gym                 17.777778
Tennis Court        15.555556
Dog Run             13.333333
Department Store    11.111111
dtype: float64

In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,"Moore Park, Summerhill East",Restaurant,Gym,Tennis Court,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


#### Cluster 5  
This cluster is also very small and both neighborhoods seem very similar

In [185]:
get_venue_freq(cluster_num=4).head()

Fast Food Restaurant    20.000000
Discount Store           8.888889
Department Store         8.888889
Sandwich Place           7.777778
Event Space              7.777778
dtype: float64

In [186]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Malvern, Rouge",Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
56,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Fast Food Restaurant,Discount Store,Sandwich Place,Doner Restaurant,Dim Sum Restaurant,Diner,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
